In [2]:
import numpy as np
import warnings
X = {}
y = {}
X["train"] = np.load("X_train.npy")
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_test.npy")
y["dev"] = np.load("y_test.npy")

In [3]:
X["train"].shape, X["dev"].shape

((18000, 300), (1000, 300))

In [4]:
(np.sum(y["train"]) + np.sum(y["dev"]))/19000

0.501578947368421

In [78]:
from sklearn.linear_model import LogisticRegression
l2 = 1
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.696

In [79]:
np.save("old_predictions.npy", pred)

In [65]:
import sklearn.metrics as mc
mc.f1_score(y["dev"], np.round(pred))

0.7013752455795678

In [66]:
fpr, tpr, thresholds = mc.roc_curve(y["dev"], pred)
mc.auc(fpr, tpr)

0.7526752675267527

In [67]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
g = np.sum(gradient_train, axis=0) / 6920
gradient_train.shape, gradient_dev.shape

((18000, 301), (1000, 301))

In [68]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [69]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1000, 18000)

In [82]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    #print("k", k)
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
    print("top_k_index", top_k_index)

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    
    
    #print("prediction", prediction)

    return X_k, y_k, prediction, top_k_index

In [84]:
_, _, _, li = Remove(500, delta_pred, 4)

top_k_index [13512 17267 17495  4141 16339  4729  6007 14885 13955 17483  5423 13608
 13945  4740  3989 10175 16542  7985  2702 17464 13498 12173  8129  8143
  7699 15893  2035 12293 13817 14675 14363 11507 12895 13312 11740  5177
  3483 10273  3459  8545  9541 15293 17915  5223 12295  8165 17835 15139
  2927  2739 11759 17966 13413 11792  4024  1703   619 14394  2596 11806
 15716 12603  8878  5568 10115  8215 10094 11669 11896 14234 17839 13781
 12398  8158 14301 16078 16488 15397 13256 15744 17101    10  6200  5890
  1014 10056  2218  5245  5418 16487 10013  4293 11235  7957 17074  8003
  7856 11547   429  2085 17094  2536  4889  6282  9771  6248 16941  4106
  1164 10237  9957 12763  1721 16046  3145  8717 10506 11456 10912  1800
  6397   105  6218 16515  2971 15347 16797  1852   115 11738  2677  9275
 12375  2577   847  5092  8151 14488  1213  7182  5593  9442 13674   162
  3165  3923 14709  8012 16207  2518  6728  1646 15081 11368  2732 11171
  6325 16283  9766  8789  8052  6314 17

In [71]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction = Remove(k, scores, dev_index)
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    new = model_k.predict_proba(test_point)[0][1]

    change = -(model.predict_proba(test_point)[0][1] - model_k.predict_proba(test_point)[0][1])
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model_k.predict(test_point))
    
    
    #print("change    ", change)
    #print("old       ", model.predict_proba(test_point)[0][1])
    #print()
    
    
    
  
    return change, flip, prediction, new

# Find approximate k by IF

In [72]:
def pred_change(k, scores, test_idx):
    X_k, y_k, prediction = Remove(k, delta_pred, test_idx)
    

In [73]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    for k in range(1, X["train"].shape[0]):
        _, _, change = Remove(k, delta_pred, test_idx)
        
        #new_train(k, test_idx, delta_pred)
        #print("cahnge", change)
        if change == None:
            return None
        
        if old > 0.5 and old + change < 0.5:
            return k
        elif old < 0.5 and old + change > 0.5:
            return k
        
    return None

In [74]:
# Loop over all dev points:

appro_ks = np.zeros((X["dev"].shape[0], 1))
new_pred = np.zeros((X["dev"].shape[0], 1))

for test_idx in range(X["dev"].shape[0]):
    appro_k = approximate_k(test_idx)
    appro_ks[test_idx] = appro_k
    if appro_k == None:
        new = None
    else:
        _, _ , _, new = new_train(appro_k, test_idx, delta_pred)
    new_pred[test_idx] = new
    print(test_idx, appro_k, pred[test_idx], new)
    print()


np.save("appro_ks.npy", appro_ks)
np.save("new_pred.npy", new_pred)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0 145 [0.28689994] 0.5975689327413284



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1 84 [0.32146259] 0.5555907386488884



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


997 47 [0.58318626] 0.48393194117370475



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


998 52 [0.57941151] 0.48608067514208675

999 178 [0.62664348] 0.46318879177920097



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [80]:
X["train"].shape

(18000, 300)

# exact_k

# Difference of change and prediction when k increasing

In [ ]:
"""
row = len(list(range(0, X["dev"].shape[0])))
col = len(list(range(1, np.sum(y["train"]))))
changes = np.zeros((row, col))
prediction = np.zeros((row, col))

import warnings
warnings.filterwarnings("ignore")
for i in range(0, X["dev"].shape[0]):
    print(i)
    for j in range(1, np.sum(y["train"]):
        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)
        
   
np.save("pred_change_app.npy", prediction)
np.save("changes.npy", changes)

difference = prediction - changes
np.save("difference_k_appk.npy", difference)
"""